In [1]:
import sys
!{sys.executable} -m pip install lxml

In [2]:
import requests
import lxml.html as lh
import pandas as pd
import plotly.graph_objs as plotly
import numpy as np
import datetime

In [20]:
#Let's see what the output for one month looks like:
year = '2022'
month = '10' #adapt for for loop
state = 'massachusetts'
location = 'great-hill-ma'

#This code (next 3 lines) is adapted from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
url='https://www.usharbors.com/harbor/'+state+'/'+location+'/tides/?tide='+year+'-'+month+'#monthly-tide-chart'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#save the raw html rows as "table"
table = tr_elements

In [23]:
url

'https://www.usharbors.com/harbor/massachusetts/great-hill-ma/tides/?tide=2022-10#monthly-tide-chart'

In [42]:
#print the lengths of the first few rows to see where the actual tide chart starts
print("\nThe number of columns in the first twelve rows are:")
print([len(T) for T in table[:]])

#check the output of one row
print("\nSample row output:")
row = 22
for col in range(len(table[row])):
    print(table[row][col].text_content())
print("Doesn't look like we need the last 3 columns. But sometimes there is an extra tide in the day, so we will grab all up to 14")
number_of_columns = 14
    
#Check how many rows we have
print("\nThe number of rows is: " + str(len(table)))


The number of columns in the first twelve rows are:
[6, 10, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 6, 10]

Sample row output:
21
Fri
10:06
4.1
10:33
3.9
3:03
0.1
3:33
-0.1
7:02
4:45

21
Fri
Doesn't look like we need the last 3 columns. But sometimes there is an extra tide in the day, so we will grab all up to 14

The number of rows is: 35


In [43]:
#Function which scrapes the URL for the current month
def get_month_table(month):
    if month > 9: #month as a string to use for the URL def
        month_str = str(month)
    else: 
        month_str = '0'+str(month)
    #This code (next 3 lines) is adapted from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
    url='https://www.usharbors.com/harbor/'+state+'/'+location+'/tides/?tide='+year+'-'+month_str+'#monthly-tide-chart'
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    #save the raw html rows as "table"
    table = tr_elements
    return(table)

In [44]:
def find_first_row(table):
    for row in range(len(table)):
        if table[row][0].text_content() == '1':
            return(row)

In [45]:
def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [46]:
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

In [105]:
year_str = str(year)

#Empty df to store the data:
df = []

for month in range(1,13):
    month_str_display = months[month-1]
    table = get_month_table(month) #get the data for this month

    #find the first row
    row = find_first_row(table)

    #while the first entry of the row is less than the number of days in the month
    while isInt(table[row][0].text_content()):
        row_data = ['']*number_of_columns
        for col in range(number_of_columns):
            row_data[col] = table[row][col].text_content()
        row_data.insert(0,month_str_display)
        row_data.insert(0,month)
        df.append(row_data)
        row+=1

In [106]:
row_data

[12,
 'December',
 '31',
 'Sat',
 '2:27',
 '4.4',
 '2:51',
 '3.7',
 '10:02',
 '0.4',
 '9:25',
 '0.4',
 '7:08',
 '4:23',
 '',
 '31']

In [107]:
print("The number of rows is "+str(len(df))+", this number should match the number of days in the given year, so 365 or 366")


The number of rows is 365, this number should match the number of days in the given year, so 365 or 366


In [138]:
df2 = []
for row in df:
    date_info = row[0:4]
    last = len(row)
    for i, x in enumerate(row):
        if x=='':
            last = i
    row = row[4:last]
    sun_info = row[-2:]
    date_info.append(sun_info[0])
    date_info.append(sun_info[1])
    row = row[:-2]
    i=0
    am_or_pm = 'AM'
    while i+1 < len(row):
        new_row = date_info.copy()
        has = bool(re.match('.*[AP]M',row[i]))
        if not(has):
            row[i] = row[i]+' '+am_or_pm
        if am_or_pm == 'AM':
            am_or_pm = 'PM'
        else:
            am_or_pm = 'AM'
        if not(bool(re.match(' [AP]M',row[i]))):
            new_row.append(row[i])
            new_row.append(row[i+1])
            df2.append(new_row)
            print(new_row)
        i += 2

[1, 'January', '1', 'Sat', '7:08', '4:24', '6:20 AM', '5.2']
[1, 'January', '1', 'Sat', '7:08', '4:24', '6:48 PM', '4.1']
[1, 'January', '1', 'Sat', '7:08', '4:24', '12:19 PM', '-0.3']
[1, 'January', '2', 'Sun', '7:08', '4:25', '7:12 AM', '5.4']
[1, 'January', '2', 'Sun', '7:08', '4:25', '7:39 PM', '4.3']
[1, 'January', '2', 'Sun', '7:08', '4:25', '12:10 AM', '-0.4']
[1, 'January', '2', 'Sun', '7:08', '4:25', '1:12 PM', '-0.3']
[1, 'January', '3', 'Mon', '7:08', '4:26', '8:02 AM', '5.4']
[1, 'January', '3', 'Mon', '7:08', '4:26', '8:30 PM', '4.4']
[1, 'January', '3', 'Mon', '7:08', '4:26', '1:04 AM', '-0.4']
[1, 'January', '3', 'Mon', '7:08', '4:26', '2:09 PM', '-0.4']
[1, 'January', '4', 'Tue', '7:08', '4:27', '8:53 AM', '5.2']
[1, 'January', '4', 'Tue', '7:08', '4:27', '9:20 PM', '4.4']
[1, 'January', '4', 'Tue', '7:08', '4:27', '2:04 AM', '-0.3']
[1, 'January', '4', 'Tue', '7:08', '4:27', '3:04 PM', '-0.4']
[1, 'January', '5', 'Wed', '7:08', '4:28', '9:43 AM', '5.0']
[1, 'January', 

In [139]:
column_names = ['MonthNum','Month','Date','Day','Sunrise','Sunset','Time','Height']
data = pd.DataFrame(df2,columns=column_names)

In [140]:
data

,MonthNum,Month,Date,Day,Sunrise,Sunset,Time,Height
0,1,January,1,Sat,7:08,4:24,6:20 AM,5.2
1,1,January,1,Sat,7:08,4:24,6:48 PM,4.1
2,1,January,1,Sat,7:08,4:24,12:19 PM,-0.3
3,1,January,2,Sun,7:08,4:25,7:12 AM,5.4
4,1,January,2,Sun,7:08,4:25,7:39 PM,4.3
...,...,...,...,...,...,...,...,...
1405,12,December,30,Fri,7:08,4:22,7:53 PM,0.3
1406,12,December,31,Sat,7:08,4:23,2:27 AM,4.4
1407,12,December,31,Sat,7:08,4:23,2:51 PM,3.7
1408,12,December,31,Sat,7:08,4:23,10:02 AM,0.4


In [142]:
data.to_csv('tide-chart-'+location+'-'+year_str+'.csv',index=True)

In [14]:
#one time imports for plotting
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

#test the plotting 
x = np.array([1, 2, 3, 4, 5])
y = np.array([1, 3, 2, 3, 1])
fig=go.Scatter(x=x, y=y, name="spline",line_shape='spline')
py.iplot([fig],filename='test')

In [15]:
high1 = pd.melt(data,id_vars=['Month','Date','Day','height_hi_1'],value_vars=['high_1'],var_name='event', value_name='time')
high2 = pd.melt(data,id_vars=['Month','Date','Day','height_hi_2'],value_vars=['high_2'],var_name='event', value_name='time')
low1 = pd.melt(data,id_vars=['Month','Date','Day','height_low_1'],value_vars=['low_1'],var_name='event', value_name='time')
low2 = pd.melt(data,id_vars=['Month','Date','Day','height_low_2'],value_vars=['low_2'],var_name='event', value_name='time')
sunrise = pd.melt(data,id_vars=['Month','Date','Day'],value_vars=['sunrise'],var_name='event', value_name='time')
sunset = pd.melt(data,id_vars=['Month','Date','Day'],value_vars=['sunset'],var_name='event', value_name='time')

In [16]:
high1 = high1.rename(columns={"height_hi_1": "height"})
high2 = high2.rename(columns={"height_hi_2": "height"})
low1 = low1.rename(columns={"height_low_1": "height"})
low2 = low2.rename(columns={"height_low_2": "height"})
sunrise['height'] = np.nan
sunset['height'] = np.nan

In [17]:
data = (pd.concat([high1, high2, low1, low2, sunrise, sunset])
         .sort_index()
         .reset_index())

In [18]:
data = data[data['time']!='']
data

,index,Month,Date,Day,height,event,time
0,0,January,1,Sat,5.2,high_1,6:20
1,0,January,1,Sat,-0.3,low_2,12:19
2,0,January,1,Sat,4.1,high_2,6:48
3,0,January,1,Sat,NaN,sunrise,7:08
4,0,January,1,Sat,NaN,sunset,4:24
...,...,...,...,...,...,...,...
2185,364,December,31,Sat,0.4,low_2,9:25
2186,364,December,31,Sat,3.7,high_2,2:51
2187,364,December,31,Sat,4.4,high_1,2:27
2188,364,December,31,Sat,NaN,sunrise,7:08


In [19]:
import re
for row in range(len(data)):
    event = data['event'].iloc[row]
    time = data['time'].iloc[row]
    if (event == 'high_1' or event == 'low_1' or event == 'sunrise'): #morning events
        if not re.search('PM',str(time)):
            time = time + ' AM'
        time = pd.to_datetime(time).strftime('%H:%M %p')
    if (event == 'high_2' or event == 'low_2' or event == 'sunset'): #evening events
        if not re.search('AM',str(time)):
            time = time + ' PM'
        time = pd.to_datetime(time).strftime('%H:%M %p')
    data['time'].iloc[row] = time

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-01 12:00:00

In [ ]:
data

In [ ]:
data['full_date'] = data['Month']+'-'+data['Date']+'-'+year+' '+data['time']
data['full_date'] = pd.to_datetime(data['full_date'], format='%B-%d-%Y %H:%M %p')

In [ ]:
data['week_num'] = np.nan
for row in range(len(data)):
    data['week_num'].iloc[row] = data['full_date'].iloc[row].isocalendar()[1]
    if data['Day'].iloc[row] == 'Sun':
        data['week_num'].iloc[row] += 1

In [ ]:
data

In [ ]:
# minutes from the previous sunday at midnight

In [ ]:
now = datetime.datetime.now() # current time
data['minutes_until'] = np.nan
for row in range(len(data)): # calculate the number of minutes until each time
    delta = data['full_date'].iloc[row] - now
    data['minutes_until'].iloc[row] = pd.Timedelta(delta).total_seconds()/60

In [ ]:
data.loc[1,:]